### 1.tf.function装饰器

In [1]:
import tensorflow as tf

In [2]:
@tf.function
def simple_nn_layer(x, y):
    return tf.nn.relu(tf.matmul(x, y))


x = tf.random.uniform((3, 3))
y = tf.random.uniform((3, 3))

simple_nn_layer(x, y)

<tf.Tensor: id=23, shape=(3, 3), dtype=float32, numpy=
array([[0.6420122 , 0.32447857, 0.87349766],
       [0.42123744, 0.23381846, 0.8582856 ],
       [0.39059502, 0.24448627, 1.1607817 ]], dtype=float32)>

In [5]:
simple_nn_layer

In [6]:
def linear_layer(x):
    return 2 * x + 1

@tf.function
def deep_net(x):
    return tf.nn.relu(linear_layer(x))

deep_net(tf.constant((1, 2, 3)))

<tf.Tensor: id=35, shape=(3,), dtype=int32, numpy=array([3, 5, 7])>

当使用tf.function注释函数时，可以像调用任何其他函数一样调用它。 它将被编译成图，这意味着可以获得更快执行，更好地在GPU或TPU上运行或导出到SavedModel

### 2.使用Python控制流程

In [8]:
@tf.function
def square_if_positive(x):
    if x > 0:
        x = x * x
    else:
        x = 0
    return x


print('square_if_positive(2) = {}'.format(square_if_positive(tf.constant(2))))
print('square_if_positive(-2) = {}'.format(square_if_positive(tf.constant(-2))))


square_if_positive(2) = 4
square_if_positive(-2) = 0


In [12]:
@tf.function
def sum_even(items):
    s = 0
    for c in items:
        if c % 2 > 0:
            continue
        s += c
    return s

sum_even(tf.constant([10, 12, 15, 20]))

<tf.Tensor: id=149, shape=(), dtype=int32, numpy=42>

In [25]:
print(tf.autograph.to_code(sum_even.python_function, experimental_optional_features=None))
from __future__ import print_function

def tf__sum_even(items):
    do_return = False
    retval_ = None
    s = 0

def loop_body(loop_vars, s_2):
    c = loop_vars
    continue_ = False
    cond = c % 2 > 0

def if_true():
    continue_ = True
    return continue_

def if_false():
    return continue_
    continue_ = ag__.if_stmt(cond, if_true, if_false)
    cond_1 = ag__.not_(continue_)

def if_true_1():
    s_1, = s_2,
    s_1 += c
    return s_1

def if_false_1():
    return s_2
    s_2 = ag__.if_stmt(cond_1, if_true_1, if_false_1)
    return s_2,
    s, = ag__.for_stmt(items, None, loop_body, (s,))
    do_return = True
    retval_ = s
    return retval_

SyntaxError: from __future__ imports must occur at the beginning of the file (<ipython-input-25-9b279de86912>, line 5)

In [27]:
@tf.function
def fizzbuzz(n):
    msg = tf.constant('')
    for i in tf.range(n):
        if tf.equal(i % 3, 0):
            msg += 'Fizz'
        elif tf.equal(i % 5, 0):
            msg += 'Buzz'
        else:
            msg += tf.as_string(i)
        msg += '\n'
    return msg


print(fizzbuzz(tf.constant(15)).numpy().decode())

Fizz
1
2
Fizz
4
Buzz
Fizz
7
8
Fizz
Buzz
11
Fizz
13
14



### 3.Keras和AutoGraph

In [29]:
class CustomModel(tf.keras.models.Model):
    
    @tf.function
    def call(self, input_data):
        if tf.reduce_mean(input_data) > 0:
            return input_data
        else:
            return input_data
        
model = CustomModel()

model(tf.constant([-2, -4]))

<tf.Tensor: id=387, shape=(2,), dtype=int32, numpy=array([-2, -4])>

In [30]:
v = tf.Variable(5)

@tf.function
def find_next_odd():
    v.assign(v + 1)
    if tf.equal(v % 2, 0):
        v.assign(v +1)
        

find_next_odd()

In [31]:
v

<tf.Variable 'Variable:0' shape=() dtype=int32, numpy=7>

### 4.用AutoGraph训练一个简单模型

In [33]:
def prepare_mnist_feature_and_labels(x, y):
    x = tf.cast(x, tf.float32) / 255.
    y = tf.cast(y, tf.int64)
    return x, y

def mnist_dataset():
    (x, y), _ = tf.keras.datasets.mnist.load_data()
    ds = tf.data.Dataset.from_tensor_slices((x, y))
    ds = ds.map(prepare_mnist_feature_and_labels)
    ds = ds.take(20000).shuffle(20000).batch(100)
    return ds

train_dataset = mnist_dataset()
model = tf.keras.Sequential((
    tf.keras.layers.Reshape(target_shape=(28 * 28,), input_shape=(28, 28)),
    tf.keras.layers.Dense(100, activation='relu'),
    tf.keras.layers.Dense(100, activation='relu'),
    tf.keras.layers.Dense(10)
))
model.build()
optimizer = tf.keras.optimizers.Adam()
compute_loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

compute_accuracy = tf.keras.metrics.SparseCategoricalCrossentropy()

def train_one_step(model, optimizer, x, y):
    with tf.GradientTape() as tape:
        logits = model(x)
        loss = compute_loss(y, logits)
        
    grads = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))
    
    compute_accuracy(y, logits)
    return loss

@tf.function
def train(model, optimizer):
    train_ds = mnist_dataset()
    step = 0
    loss = 0.0
    accuracy = 0.0
    for x, y in train_ds:
        step += 1
        loss = train_one_step(model, optimizer, x, y)
        if tf.equal(step % 10, 0):
            tf.print('Step', step, ': loss', loss, '; accuracy', compute_accuracy.result())
    return step, loss, accuracy


            
step, loss, accuracy = train(model, optimizer)
print('Final step', step, ': loss', loss, '; accuracy', compute_accuracy.result())

Step 10 : loss 1.84710193 ; accuracy 5.03456163
Step 20 : loss 1.17040598 ; accuracy 3.31952095
Step 30 : loss 0.956142187 ; accuracy 2.68687534
Step 40 : loss 0.677743316 ; accuracy 2.39112306
Step 50 : loss 0.413177371 ; accuracy 2.19970369
Step 60 : loss 0.490855038 ; accuracy 2.06108212
Step 70 : loss 0.6381796 ; accuracy 1.97479367
Step 80 : loss 0.391855747 ; accuracy 1.90126574
Step 90 : loss 0.443323821 ; accuracy 1.84079015
Step 100 : loss 0.383209139 ; accuracy 1.78723335
Step 110 : loss 0.414580703 ; accuracy 1.74700642
Step 120 : loss 0.231030375 ; accuracy 1.70875764
Step 130 : loss 0.296536416 ; accuracy 1.67705667
Step 140 : loss 0.272495985 ; accuracy 1.64435148
Step 150 : loss 0.285137951 ; accuracy 1.61732221
Step 160 : loss 0.181327492 ; accuracy 1.59544671
Step 170 : loss 0.237601757 ; accuracy 1.57168353
Step 180 : loss 0.390198201 ; accuracy 1.55919564
Step 190 : loss 0.25938195 ; accuracy 1.54355955
Step 200 : loss 0.264018327 ; accuracy 1.5299679
Final step tf.T